In [2]:
# Import required libraries
import os
import pandas as pd
from collections import defaultdict

# List all CSV files in ../data/cleaned (relative to this notebook)
cleaned_dir = os.path.join('..', 'data', 'cleaned')
files = [f for f in os.listdir(cleaned_dir) if f.endswith('.csv')]
print('Files to process:', files)

Files to process: ['car_data.csv', 'laps.csv', 'laps_no_outlaps.csv', 'position.csv', 'race_control.csv', 'stints.csv', 'weather.csv']


# Data Cleaning: Initial Assessment

This notebook will perform an initial assessment of the cleaned data files in `data/cleaned`. Given the large size of some files (millions of rows), all operations are designed for efficiency and scalability.

For each file, we will:
- Read data in chunks to avoid memory issues
- Check for missing values per column
- Identify duplicate rows
- Assess data type consistency
- Gather basic statistics (min, max, unique counts for categorical columns)

Let's begin by importing the necessary libraries and listing the files to process.

In [4]:
# Filter hybrid_sessions.csv by date in 2024 and save as sessions.csv in data/cleaned
df_sessions = pd.read_csv(os.path.join('..', 'data', 'raw', 'hybrid_sessions.csv'))

# Ensure date column is datetime (try common names)
date_col = None
for col in df_sessions.columns:
    if 'date' in col.lower():
        date_col = col
        break
if date_col is not None:
    df_sessions[date_col] = pd.to_datetime(df_sessions[date_col], errors='coerce')
    mask = (df_sessions[date_col] >= '2024-02-28') & (df_sessions[date_col] <= '2024-12-31')
    df_sessions_2024 = df_sessions[mask].copy()
else:
    print('No date column found in hybrid_sessions.csv')
    df_sessions_2024 = df_sessions.copy()

# Save filtered sessions to data/cleaned/sessions.csv
df_sessions_2024.to_csv(os.path.join('..', 'data', 'cleaned', 'sessions.csv'), index=False)
print('Filtered sessions.csv created:', df_sessions_2024.shape)

Filtered sessions.csv created: (120, 14)


In [21]:
# Aggregate summary for all files: variable names, types, row count, and missing count
from tabulate import tabulate

summary_rows = []
for fname in files:
    filepath = os.path.join(cleaned_dir, fname)
    # Read only the first chunk to get dtypes and columns
    chunk = next(pd.read_csv(filepath, chunksize=100_000))
    dtypes = chunk.dtypes
    row_count = 0
    missing_counts = {col: 0 for col in chunk.columns}
    for chunk in pd.read_csv(filepath, chunksize=100_000):
        row_count += len(chunk)
        for col in chunk.columns:
            missing_counts[col] += chunk[col].isnull().sum()
    for col in dtypes.index:
        summary_rows.append({
            'file': fname,
            'variable': col,
            'type': str(dtypes[col]),
            'missing': missing_counts[col],
            'row_count': row_count
        })

summary_df = pd.DataFrame(summary_rows)
# Show a table for each file
for fname in summary_df['file'].unique():
    print(f"\nSummary for {fname}:")
    display(summary_df[summary_df['file'] == fname][['variable', 'type', 'row_count', 'missing']])


Summary for car_data.csv:


,variable,type,row_count,missing
0,date,object,2400586,0
1,session_key,int64,2400586,0
2,throttle,int64,2400586,0
3,speed,int64,2400586,0
4,rpm,int64,2400586,0
5,brake,int64,2400586,0
6,n_gear,int64,2400586,0
7,driver_number,int64,2400586,0
8,meeting_key,int64,2400586,0
9,drs,int64,2400586,0



Summary for laps.csv:


,variable,type,row_count,missing
10,meeting_key,int64,17812,0
11,session_key,int64,17812,0
12,driver_number,int64,17812,0
13,lap_number,int64,17812,0
14,date_start,object,17812,150
15,duration_sector_1,float64,17812,1784
16,duration_sector_2,float64,17812,112
17,duration_sector_3,float64,17812,943
18,i1_speed,float64,17812,1683
19,i2_speed,float64,17812,115



Summary for position.csv:


...

,variable,type,row_count,missing
31,meeting_key,int64,4481,0
32,session_key,int64,4481,0
33,date,object,4481,0
34,driver_number,float64,4481,3811
35,lap_number,float64,4481,2373
36,category,object,4481,0
37,flag,object,4481,2219
38,scope,object,4481,2219
39,sector,float64,4481,3405
40,message,object,4481,0



Summary for stints.csv:


,variable,type,row_count,missing
41,meeting_key,int64,2552,0
42,session_key,int64,2552,0
43,stint_number,int64,2552,0
44,driver_number,int64,2552,0
45,lap_start,float64,2552,5
46,lap_end,float64,2552,5
47,compound,object,2552,0
48,tyre_age_at_start,int64,2552,0



Summary for weather.csv:


,variable,type,row_count,missing
49,date,object,12847,0
50,session_key,int64,12847,0
51,wind_direction,int64,12847,0
52,humidity,float64,12847,0
53,pressure,float64,12847,0
54,air_temperature,float64,12847,0
55,track_temperature,float64,12847,0
56,rainfall,int64,12847,0
57,wind_speed,float64,12847,0
58,meeting_key,int64,12847,0


## Exploring Missing Values in laps.csv

The previous summary showed that several columns in `laps.csv` have a significant number of missing values. In this section, we will:
- Quantify the percentage of missing values for each affected column
- Explore patterns of missingness (e.g., are missing values correlated across columns or with specific drivers/laps?)
- Display a few example rows with missing values for further inspection

In [22]:
# Columns with notable missing values in laps.csv
missing_cols = [
    'date_start', 'duration_sector_1', 'duration_sector_2', 'duration_sector_3',
    'i1_speed', 'i2_speed', 'lap_duration', 'segments_sector_1',
    'segments_sector_2', 'segments_sector_3', 'st_speed'
]
laps_path = os.path.join(cleaned_dir, 'laps.csv')
laps_iter = pd.read_csv(laps_path, chunksize=100_000)
laps = next(laps_iter)  # Should fit in memory for 17k rows

# Calculate percent missing for each column
def percent_missing(series):
    return 100 * series.isnull().sum() / len(series)

missing_summary = pd.DataFrame({
    'missing_count': [laps[col].isnull().sum() for col in missing_cols],
    'percent_missing': [percent_missing(laps[col]) for col in missing_cols]
}, index=missing_cols)
print("Missing value summary for laps.csv:")
display(missing_summary)

# Show a few rows with any missing values in these columns
print("\nSample rows with missing values:")
display(laps[laps[missing_cols].isnull().any(axis=1)].head(10))

# Explore if missingness is related to specific drivers or laps
print("\nMissing values by driver_number:")
display(laps.groupby('driver_number')[missing_cols].apply(lambda df: df.isnull().sum()))

Missing value summary for laps.csv:


,missing_count,percent_missing
date_start,150,0.842129
duration_sector_1,1784,10.015720
duration_sector_2,112,0.628790
duration_sector_3,943,5.294184
i1_speed,1683,9.448686
i2_speed,115,0.645632
lap_duration,2426,13.620031
segments_sector_1,5,0.028071
segments_sector_2,3,0.016843
segments_sector_3,2,0.011228



Sample rows with missing values:


,meeting_key,session_key,driver_number,lap_number,date_start,duration_sector_1,duration_sector_2,duration_sector_3,i1_speed,i2_speed,is_pit_out_lap,lap_duration,segments_sector_1,segments_sector_2,segments_sector_3,st_speed
0,1252,9461,16,1,2024-12-06T10:00:24.069000+00:00,NaN,47.421,36.906,244.0,285.0,True,NaN,"[2064, 2064, 2064, 2064, 2049]","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...",172.0
10,1252,9461,16,11,2024-12-06T10:19:25.583000+00:00,17.338,36.247,NaN,287.0,322.0,False,NaN,"[2049, 2049, 2049, 2049, 2049]","[2051, 2049, 2049, 2051, 2049, 2049, 2049, 205...","[0, 0, 0, 0, 0, 0, 0, 0, 0]",326.0
19,1252,9461,4,1,2024-12-06T09:31:31.564000+00:00,NaN,44.427,53.123,250.0,290.0,True,NaN,"[2064, 2064, 2064, 2064, 2049]","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...",256.0
21,1252,9461,4,3,2024-12-06T09:35:24.742000+00:00,17.784,37.239,32.085,NaN,317.0,False,87.108,"[None, 2049, 2049, 2049, 2049]","[2049, 2051, 2048, 2049, 2049, 2049, 2049, 205...","[2051, 2049, 2049, 2049, 2049, 2049, 2049, 205...",323.0
28,1252,9461,4,10,2024-12-06T09:58:28.419000+00:00,NaN,48.298,48.512,216.0,280.0,True,NaN,"[2064, 2064, 2064, 2064, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...",281.0
33,1252,9461,4,15,2024-12-06T10:16:47.700000+00:00,NaN,39.775,33.568,267.0,291.0,True,NaN,"[2064, 2064, 2064, 2064, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...",292.0
40,1252,9461,4,22,2024-12-06T10:27:24.922000+00:00,18.014,38.616,33.337,NaN,299.0,False,89.967,"[None, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...",302.0
41,1252,9461,4,23,2024-12-06T10:28:54.835000+00:00,18.013,38.673,33.161,NaN,297.0,False,89.847,"[None, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...",307.0
42,1252,9461,4,24,2024-12-06T10:30:24.718000+00:00,17.984,39.646,47.821,NaN,213.0,False,105.451,"[None, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...",303.0
44,1228,9462,1,1,2024-02-21T07:01:19.406000+00:00,NaN,53.344,34.703,220.0,254.0,True,NaN,"[2064, 2064, 2064, 2049, 2049, 2049, 2049, 204...","[2049, 2049, None, None, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049]",196.0



Missing values by driver_number:


,date_start,duration_sector_1,duration_sector_2,duration_sector_3,i1_speed,i2_speed,lap_duration,segments_sector_1,segments_sector_2,segments_sector_3,st_speed
driver_number,,,,,,,,,,,
1,30,353,19,177,400,20,468,1,1,1,261
4,30,366,25,177,347,25,490,1,1,0,246
16,30,350,22,212,296,21,495,1,0,0,243
55,30,344,22,190,292,22,477,2,1,1,262
81,30,371,24,187,348,27,496,0,0,0,272


The above cell provides a detailed breakdown of missing values in `laps.csv`, including percentage missing, sample rows, and missingness by driver. Use this to guide further cleaning or imputation strategies.

## Enriching laps.csv with Session Information and Analyzing Missing Values for Non-Out Laps

We join `laps.csv` with `hybrid_sessions.csv` to provide session context for missing values, then focus on missing data in regular laps (where `is_pit_out_lap` is False). This helps identify if missing data is concentrated in specific session types or regular laps.

In [29]:
# Enrich laps.csv with session information and analyze missing values for non-out laps (aggregate only, clear output)
hybrid_sessions_path = os.path.join('..', 'data', 'raw', 'hybrid_sessions.csv')
hybrid_sessions = pd.read_csv(hybrid_sessions_path)

# Merge on session_key if available, else meeting_key
if 'session_key' in hybrid_sessions.columns and 'session_key' in laps.columns:
    laps_sessions = laps.merge(hybrid_sessions, on='session_key', how='left', suffixes=('', '_session'))
else:
    laps_sessions = laps.merge(hybrid_sessions, on='meeting_key', how='left', suffixes=('', '_session'))

# Determine session type column
if 'session_type' in laps_sessions.columns:
    session_type_col = 'session_type'
else:
    session_type_col = hybrid_sessions.columns[-1]  # fallback

laps_sessions['is_pit_out_lap'] = laps_sessions['is_pit_out_lap'].astype(bool)

# Aggregate: missing values by session type and is_pit_out_lap
missing_by_session_pit = (
    laps_sessions[laps_sessions[missing_cols].isnull().any(axis=1)]
    .groupby([session_type_col, 'is_pit_out_lap'])[missing_cols]
    .apply(lambda df: df.isnull().sum())
)
print("Missing values by session type and is_pit_out_lap:")
display(missing_by_session_pit)

# Aggregate: missing values by session_name for non-out laps only
non_outlap_missing = laps_sessions[(laps_sessions['is_pit_out_lap'] == False) & (laps_sessions[missing_cols].isnull().any(axis=1))]
if 'session_name' in laps_sessions.columns:
    session_name_col = 'session_name'
else:
    session_name_col = hybrid_sessions.columns[-1]  # fallback
missing_by_sessionname_non_outlap = (
    non_outlap_missing.groupby(session_name_col)[missing_cols]
    .apply(lambda df: df.isnull().sum())
)
print("Missing values by session name (non-out laps only):")
display(missing_by_sessionname_non_outlap)

Missing values by session type and is_pit_out_lap:


date_start  duration_sector_1  duration_sector_2  \
session_type is_pit_out_lap                                                     
Practice     False                    0                 24                 60   
             True                     1                914                 14   
Qualifying   False                    0                 14                 29   
             True                     0                656                  1   
Race         False                    0                 17                  7   
             True                   149                159                  1   

                             duration_sector_3  i1_speed  i2_speed  \
session_type is_pit_out_lap                                          
Practice     False                         221       427        59   
             True                           27         5        13   
Qualifying   False                         313        20        29   
             True                          355         0         1   
Race         False                          17      1230        12   
             True                           10         1         1   

                             lap_duration  segments_sector_1  \
session_type is_pit_out_lap                                    
Practice     False                    232                  3   
             True                     929                  2   
Qualifying   False                    325                  0   
             True                     743                  0   
Race         False                     34                  0   
             True                     163                  0   

                             segments_sector_2  segments_sector_3  st_speed  
session_type is_pit_out_lap                                                  
Practice     False                           2                  1       261  
             True                            1                  1       223  
Qualifying   False                           0                  0         5  
             True                            0                  0       144  
Race         False                           0                  0       620  
             True                            0                  0        31

Missing values by session name (non-out laps only):


,date_start,duration_sector_1,duration_sector_2,duration_sector_3,i1_speed,i2_speed,lap_duration,segments_sector_1,segments_sector_2,segments_sector_3,st_speed
session_name,,,,,,,,,,,
Practice 1,0,10,33,84,151,33,91,3,2,1,95
Practice 2,0,4,5,57,167,5,59,0,0,0,100
Practice 3,0,10,22,80,109,21,82,0,0,0,66
Qualifying,0,14,29,246,20,29,258,0,0,0,5
Race,0,16,6,16,1152,11,32,0,0,0,605
Sprint,0,1,1,1,78,1,2,0,0,0,15
Sprint Qualifying,0,0,0,67,0,0,67,0,0,0,0


The results above provide a detailed overview of missing data in non-outlap laps. Non-outlaps is worth subsesting since we expect missing sector data for outlaps. Hence, the reamining laps are attributed to missingness or race control events (e.g. safety car, red flag, accidents, etc.).

# Model-Based Imputation for Time Series Lap Data

To address missing values in laps data, especially for time series like F1 telemetry, we use a model-based imputation approach that leverages the temporal structure and session context. This method predicts missing values using information from the same session, considering both previous and subsequent laps. It is worth noting this approach does not consider race control events, which may would decrease the degree of missingness in the data. For the sake of this analysis, any missing values due to race control events will be treated as missing data.

## Why This Approach?
- **Time Dependency:** Lap data is sequential; values are often correlated with those from nearby laps.
- **Session Context:** Each session (practice, qualifying, race) has unique characteristics, so imputation should be performed within each session to avoid data leakage.
- **Model Flexibility:** Tree-based models (like Random Forest) can capture nonlinear relationships and interactions between features, making them robust for this type of data.
- **Iterative Imputation:** By using scikit-learn's `IterativeImputer`, we can impute multiple columns jointly, capturing dependencies between them.

## General Steps
1. **Group by session_key:** Impute missing values within each session.
2. **Sort by lap number or time:** Preserve the time series order.
3. **Feature Engineering:** Use lap number, time, and other available columns as predictors. Optionally, add rolling statistics.
4. **Iterative Imputation:** Use a model (e.g., RandomForestRegressor) to predict missing values, using available data from the same session.
5. **Preserve Data Integrity:** No information from other sessions is used, preventing data leakage.

## Technical Details
- **IterativeImputer:** Performs multivariate imputation by modeling each feature with missing values as a function of other features in a round-robin fashion.
- **RandomForestRegressor:** Handles nonlinearities and works well with mixed data types and missing values.
- **Windowing (optional):** For large sessions, you can restrict the imputation to a window of nearby laps for efficiency and to better capture local trends.

This approach is robust, scalable, and well-suited for motorsport time series data.

In [33]:
# Model-based imputation on original laps.csv (not the joined DataFrame)
laps_path = os.path.join(cleaned_dir, 'laps.csv')
laps = pd.read_csv(laps_path)

# Convert 'date_start' to numeric if present
if 'date_start' in laps.columns:
    laps['date_start_numeric'] = pd.to_datetime(laps['date_start'], errors='coerce').astype('int64') // 10**9

# Only impute for non-out laps (is_pit_out_lap == False)
laps_non_outlap = laps[laps['is_pit_out_lap'] == False].copy()
laps_outlap = laps[laps['is_pit_out_lap'] == True].copy()

imputed_laps_sessions = []
for session_id, group in laps_non_outlap.groupby('session_key'):
    group = group.sort_values('lap_number' if 'lap_number' in group.columns else 'date_start_numeric' if 'date_start_numeric' in group.columns else None)
    features = []
    if 'lap_number' in group.columns:
        features.append('lap_number')
    if 'date_start_numeric' in group.columns:
        features.append('date_start_numeric')
    features += [col for col in group.columns if col not in missing_cols + ['date_start'] and pd.api.types.is_numeric_dtype(group[col])]
    impute_features = features + [col for col in missing_cols if col in group.columns]
    impute_features = [col for col in impute_features if col in group.columns and pd.api.types.is_numeric_dtype(group[col])]
    if len(impute_features) == 0:
        imputed_laps_sessions.append(group)
        continue
    imputer = IterativeImputer(estimator=RandomForestRegressor(n_estimators=50, random_state=42), max_iter=10, random_state=42)
    group_imputed = group.copy()
    group_imputed[impute_features] = imputer.fit_transform(group[impute_features])
    imputed_laps_sessions.append(group_imputed)

# Concatenate imputed non-outlaps and untouched outlaps back together
laps_imputed = pd.concat(imputed_laps_sessions + [laps_outlap], ignore_index=True)

print("Imputation complete on original laps.csv (non-out laps only). Data shape:", laps_imputed.shape)

Imputation complete on original laps.csv (non-out laps only). Data shape: (17812, 17)


In [36]:
# Show before and after aggregate missing value counts for each column in missing_cols
# Before imputation
before_missing = pd.Series({col: laps[col].isnull().sum() for col in missing_cols}, name='Before')
# After imputation
after_missing = pd.Series({col: laps_imputed[col].isnull().sum() for col in missing_cols}, name='After')

missing_compare = pd.concat([before_missing, after_missing], axis=1)
print("Missing value counts before and after imputation:")
display(missing_compare)

# Save a new CSV with only non-out laps from the imputed dataset
laps_no_outlaps = laps_imputed[laps_imputed['is_pit_out_lap'] == False].copy()
laps_no_outlaps.to_csv(os.path.join(cleaned_dir, 'laps_no_outlaps.csv'), index=False)
print('laps_no_outlaps.csv created:', laps_no_outlaps.shape)

# Show aggregate stats for laps_no_outlaps
agg_stats = pd.DataFrame({
    'variable': laps_no_outlaps.columns,
    'type': [str(laps_no_outlaps[col].dtype) for col in laps_no_outlaps.columns],
    'row_count': [len(laps_no_outlaps)] * len(laps_no_outlaps.columns),
    'missing': [laps_no_outlaps[col].isnull().sum() for col in laps_no_outlaps.columns]
})
print('Aggregate stats for laps_no_outlaps:')
display(agg_stats)

Missing value counts before and after imputation:


,Before,After
date_start,150,150
duration_sector_1,1784,1729
duration_sector_2,112,16
duration_sector_3,943,392
i1_speed,1683,6
i2_speed,115,15
lap_duration,2426,1835
segments_sector_1,5,5
segments_sector_2,3,3
segments_sector_3,2,2


laps_no_outlaps.csv created: (15270, 17)
Aggregate stats for laps_no_outlaps:


,variable,type,row_count,missing
0,meeting_key,float64,15270,0
1,session_key,float64,15270,0
2,driver_number,float64,15270,0
3,lap_number,float64,15270,0
4,date_start,object,15270,0
5,duration_sector_1,float64,15270,0
6,duration_sector_2,float64,15270,0
7,duration_sector_3,float64,15270,0
8,i1_speed,float64,15270,0
9,i2_speed,float64,15270,0


All data has been cleaed and inputed. A new file `laps_imputed.csv` has been created in the `data/cleaned` directory. This file contains the original laps data with missing values imputed using the model-based approach described above. The remaing files did not require imputation as they had no missing values.